# Fundamental matrix estimation with calibrated cameras

In case we have a stereo-vision system calibrated, then:

\begin{equation}
\mathbf{l}' = (\mathbf{P'C}) \times (\mathbf{P'P^+x}) = (\mathbf{e'}) \times (\mathbf{P'P^+x}) = 
[\mathbf{e'}]_\mathsf{x}(\mathbf{P'P^+})\mathbf{x} \enspace,
\end{equation}

where

\begin{equation}
\mathbf{F} =  [\mathbf{e'}]_\times(\mathbf{P'P^+}) \enspace.
\end{equation}

If

\begin{equation}
\mathbf{P} = \mathbf{K}[\mathbf{I} \,\, | \,\, \mathbf{0}] \enspace,\enspace 
\mathbf{P}' = \mathbf{K'}[\mathbf{R} \,\, | \,\, \mathbf{t}] \enspace ,
\end{equation}

then

\begin{equation}
\mathbf{P^+} =  \begin{bmatrix} \mathbf{K}^{-1} \\ \mathbf{0}^\mathsf{T} \end{bmatrix} \enspace,\enspace
\mathbf{C} = \begin{bmatrix} \mathbf{0} \\ 1 \end{bmatrix} \enspace,
\end{equation}

and

\begin{equation}
\mathbf{F} =  
[\mathbf{P'C}]_\times(\mathbf{P'P^+}) = 
[\mathbf{K't}]_\times(\mathbf{K'RK}^{-1}) = 
\mathbf{K'}^\mathsf{-T} [\mathbf{t}]_\times \mathbf{RK}^{-1} = 
\mathbf{K'}^\mathsf{-T}\mathbf{R} [\mathbf{R}^\mathsf{T}\mathbf{t}]_\times \mathbf{K}^{-1} = 
\mathbf{K'}^\mathsf{-T}\mathbf{R} \mathbf{K}^\mathsf{T} [\mathbf{KR}^\mathsf{T}\mathbf{t}]_\times
\enspace.
\end{equation}

Note that the epipoles $\mathbf{e}=\mathbf{PC'}$ and $\mathbf{e'}=\mathbf{P'C}$ are the projection of the camera centers which are the null space of the projection matrices: $\mathbf{PC}=\mathbf{0}$, $\mathbf{P'C'}=\mathbf{0}$. And using the aforementione cameras:

\begin{equation}
\mathbf{e} =  \mathbf{PC'} =  \mathbf{P}\begin{bmatrix} \mathbf{-R}^\mathsf{T}\mathbf{t} \\ 1 \end{bmatrix} = 
\mathbf{K}\mathbf{R}^\mathsf{T}\mathbf{t} \enspace,
\end{equation}

\begin{equation}
\mathbf{e'} =  \mathbf{P'C} =  \mathbf{P'}\begin{bmatrix} \mathbf{0} \\ 1 \end{bmatrix} = 
\mathbf{K't} \enspace.
\end{equation}

In this case we will use a dataset of two cameras with their intrinsic and extrinsic parameters. With that, we will build the projection matrices $\mathbf{P}$ and $\mathbf{P'}$.

In [1]:
import numpy as np

K1 = np.array([[2484.5998668515413, 0.0, 664.9167823011818],
               [0.0, 2480.5899367010766, 555.9529278857822],
               [0.0, 0.0, 1.0]])

K2 = np.array([[2537.573016481479, 0.0, 666.1235386424721],
               [0.0, 2542.869507969575, 548.2286710416115],
               [0.0, 0.0, 1.0]])

R = np.array([[0.7620802928656095, -0.013008933803163055, 0.6473518323656556],
              [0.009577298865745784, 0.9999152444573334, 0.008819254405423891],
              [-0.6474116948065606, -0.0005210980102877084, 0.7621402927837897]])

t = np.array([[-343.84806370082106],
              [0.3708675317457493],
              [145.25593928526436]])

With this information, the projection matrices are given by

In [2]:
# Create projection matrices of camera 1 and camera 2
P1 = K1 @ np.c_[np.eye(3), np.zeros(3)] # P = K[I | 0]
P2 = K2 @ np.c_[R, t] # P' = K'[R | t]

We also know the real fundamental matrix of this stereo-system given from the stereo camera calibration process. The idea is to compare the estimated fundamental matrix wiht the real matrix given from the calibration.

In [3]:
F = np.array([[1.5232273506285586e-08, 1.3584377807984262e-06, -0.0007421923846671951],
              [1.042849662001024e-06, 1.9308822417669677e-08, -0.008948446155231224],
              [-0.0004971383968785172, 0.007244417097561759, 1.0]])

## First approach

First, let's estimate the fundamental matrix with the expression: $\mathbf{F} = [\mathbf{P'C}]_\times(\mathbf{P'P^+})$. For that we need the epipole $\mathbf{e'} = \mathbf{P'C}$, which is the projection of the optical center of camera 1 into image plane 2. The camera center of camera 1 is the null-space of the projection matrix of the first view, that is, $\mathbf{PC} = \mathbf{0}$. 

As in this case, our camera 1 is also our (world frame) absolute coordinate system (due to first view extrinsics are $[\mathbf{I} \,\, | \,\, \mathbf{0}]$), we have:

$$
\mathbf{C} = \begin{bmatrix} 0\\0\\0\\1 \end{bmatrix} \enspace.
$$

Anyway, we can verify this by calculating the null space

In [4]:
from scipy.linalg import null_space

C1 = null_space(P1)
print('C = \n{}'.format(C1))

C = 
[[0.]
 [0.]
 [0.]
 [1.]]


Then, the epipole $\mathbf{e'} = \mathbf{P'C}$ is

In [5]:
e2 = P2 @ C1
print('e\' = \n{}'.format(e2))

e' = 
[[-7.75781168e+05]
 [ 8.05765383e+04]
 [ 1.45255939e+02]]


For built the skew symmetric matrix $[\mathbf{e'}]_\times = [\mathbf{P'C}]_\times$, we will use the following function:

In [6]:
def skew(x):
    x = x.flatten()
    return np.array([[0, -x[2], x[1]],
                     [x[2], 0, -x[0]],
                     [-x[1], x[0], 0]])

Then, from $\mathbf{F} = [\mathbf{P'C}]_\times(\mathbf{P'P^+}) = [\mathbf{e'}]_\times(\mathbf{P'P^+})$, we have

In [7]:
F1 = skew(e2) @ P2 @ np.linalg.pinv(P1)
print('F1 = \n{}'.format(F1))

F1 = 
[[-1.66952044e+00 -1.48890423e+02  8.13473679e+04]
 [-1.14300654e+02 -2.11632713e+00  9.80786864e+05]
 [ 5.44884331e+04 -7.94018202e+05 -1.09604153e+08]]


If you compare this matrix with the real fundamental matrix you can note that they are not equal. The fundamental matrix is a singular matrix ($\det \mathbf{F} = 0$) of rank 2. We can evaluate if our estimated matrix is singular and check its rank

In [8]:
print('det F1 = {}'.format(np.linalg.det(F1)))
print('rank F1 = {}'.format(np.linalg.matrix_rank(F1)))

det F1 = -0.000427903918985845
rank F1 = 2


Comparing both conditions with the true fundamental matrix `F`:

In [9]:
print('det F = {}'.format(np.linalg.det(F)))
print('rank F = {}'.format(np.linalg.matrix_rank(F)))

det F = -3.478047271579269e-28
rank F = 2


And as you can see the singularty condition is better in the real fundamental matrix `F`. To improve this in `F1`, we can divide the matrix with the value located in the last row, last column. As you can see in `F`, this element is 1 and in `F1` is a number different to 1.

In [10]:
F1 = F1/F1[-1,-1]
print('The new F1 = \n{}'.format(F1))

The new F1 = 
[[ 1.52322735e-08  1.35843778e-06 -7.42192385e-04]
 [ 1.04284966e-06  1.93088224e-08 -8.94844616e-03]
 [-4.97138397e-04  7.24441710e-03  1.00000000e+00]]


And you can note that this matrix is similar to `F`. Evaluating the condition of a fundamental matrix:

In [11]:
print('det F1 = {}'.format(np.linalg.det(F1)))

det F1 = 5.934373187162705e-28


We can also enforce the singularity constraint by minimizing the Frobenius norm $||\mathbf{F}-\mathbf{F'}||$ subject to $\det \mathbf{F'}=0$. Using the singular value decomposition (SVD) of $\mathbf{F}$, where $\mathbf{F} = \mathbf{UDV}^\mathsf{T}$ and $\mathbf{D}=\mathop{\mathrm{diag}}(r,s,t)$, the matrix $\mathbf{F'}=\mathbf{U}\mathop{\mathrm{diag}}(r,s,0)\,\mathbf{V}^\mathsf{T}$ minimizes the Frobenius norm of $\mathbf{F}-\mathbf{F'}$

In [12]:
u, s, vh = np.linalg.svd(F1)
F1n = u @ np.diag(np.r_[s[:2],0]) @ vh
print('det F1n = {}'.format(np.linalg.det(F1n)))

det F1n = -7.676284602201382e-29


## Second approach

In this case we will use the expression: $\mathbf{F} = \mathbf{K'}^\mathsf{-T} [\mathbf{t}]_\times \mathbf{RK}^{-1}$

In [13]:
F2 = np.linalg.inv(K2).T @ skew(t) @ R @ np.linalg.inv(K1)
print('F2 = \n{}'.format(F2))

F2 = 
[[-2.58731388e-07 -2.30740666e-05  1.26066845e-02]
 [-1.77135699e-05 -3.27974575e-07  1.51995951e-01]
 [ 8.44426195e-03 -1.23051762e-01 -1.69857368e+01]]


Where the singularity constraint

In [14]:
print('det F2 = {}'.format(np.linalg.det(F2)))
print('rank F2 = {}'.format(np.linalg.matrix_rank(F2)))

det F2 = -1.2123704645704422e-23
rank F2 = 2


In the same way as with `F1` we need to force the 1 value in the last row, last column of the matrix:

In [15]:
F2 = F2/F2[-1,-1]
print('The new F2 = \n{}'.format(F2))

The new F2 = 
[[ 1.52322735e-08  1.35843778e-06 -7.42192385e-04]
 [ 1.04284966e-06  1.93088224e-08 -8.94844616e-03]
 [-4.97138397e-04  7.24441710e-03  1.00000000e+00]]


And now the condition:

In [16]:
print('det F2 = {}'.format(np.linalg.det(F2)))

det F2 = 1.6808421263829038e-27


## Using bilinear relations

As an alternative, we can estimate the fundamental matrix based on our projection matrices using determiants with the expression:

\begin{equation}
\label{dets}
F_{ji} = (-1)^{i+j} \det \begin{bmatrix} \sim \mathbf{p}^i \\ \sim \mathbf{p'}^j \end{bmatrix}
\end{equation}

where $\mathbf{p}^i$ and $\mathbf{p'}^j$ represent the $i$-th and $j$-th rows, respectively, of the projection matrices as row vectors, and $\sim \mathbf{p}^i$ denote the matrix obtained by omitting the row $\mathbf{p}^i$. In that way we have:

In [17]:
F3 = np.zeros([3,3])
for i in range(3):
    for j in range(3):
        F3[j,i] = (-1)**(i+j)*np.linalg.det(np.r_[np.delete(P1,i,0), np.delete(P2,j,0)])

print('F3 = \n{}'.format(F3))

F3 = 
[[ 1.02897110e+07  9.17652386e+08 -5.01366071e+11]
 [ 7.04466185e+08  1.30435028e+07 -6.04485762e+12]
 [-3.35827112e+11  4.89375129e+12  6.75520365e+14]]


and the conditions

In [18]:
print('det F3 = {}'.format(np.linalg.det(F3)))
print('rank F3 = {}'.format(np.linalg.matrix_rank(F3)))

det F3 = -1.5861294258582344e+19
rank F3 = 2


Normalizing the matrix

In [19]:
F3 = F3/F3[-1,-1]
print('The new F3 = \n{}'.format(F3))

print('\ndet F3 = {}'.format(np.linalg.det(F3)))

The new F3 = 
[[ 1.52322735e-08  1.35843778e-06 -7.42192385e-04]
 [ 1.04284966e-06  1.93088224e-08 -8.94844616e-03]
 [-4.97138397e-04  7.24441710e-03  1.00000000e+00]]

det F3 = -5.1536606982749383e-26


Minimizing the Frobenius norm

In [20]:
u, s, vh = np.linalg.svd(F3)
F3n = u @ np.diag(np.r_[s[:2],0]) @ vh
print('det F3n = {}'.format(np.linalg.det(F3n)))

det F3n = 2.1359666716413443e-27


## Evaluating the epipolar constraint

We will use a set of three corresponding points $\mathbf{x} \leftrightarrow \mathbf{x'}$:

In [21]:
x1 = np.array([[463.9591120881323, 518.7725918740775, 394.78206729900245],
               [560.717542605583, 360.22058392539236, 326.45670375262875],
               [1.0, 1.0, 1.0]])

x2 = np.array([[356.69235663194286, 383.18185757215105, 279.1697577935336],
               [572.4233800180717, 387.2419002117616, 361.05521184314],
               [1.0, 1.0, 1.0]])

to evaluate the epipolar constraint $\mathbf{x'}^\mathsf{T}\mathbf{Fx}=0$ with the true fundamental matrix and the estimated with the different methods.

Using the true fundamental matrix:

In [22]:
print(np.diag(x2.T @ F @ x1))

[0.00175766 0.00480353 0.00705065]


Using the matrix calculated with the expression $\mathbf{F} = [\mathbf{P'C}]_\times(\mathbf{P'P^+})$

In [23]:
print(np.diag(x2.T @ F1 @ x1))

[0.00175766 0.00480353 0.00705065]


Using the fundamental matrix obtained with $\mathbf{F} = \mathbf{K'}^\mathsf{-T} [\mathbf{t}]_\times \mathbf{RK}^{-1}$

In [24]:
print(np.diag(x2.T @ F2 @ x1))

[0.00175766 0.00480353 0.00705065]


The results with the matrix estimated with the determinants of the projection matrices (Eq. \ref{dets})

In [25]:
print(np.diag(x2.T @ F3 @ x1))

[0.00175766 0.00480353 0.00705065]
